In [11]:
import pandas as pd
import requests
import time
from pathlib import Path

In [12]:
MODEL = "model-1"
EVAL_FILE = "../../data/processed/bdm-corpus-2/stage-5/test_filtered.csv"
RESULT_FILE = "../../results/exp002/results_filtered.csv"
TEMPERATURE = 0.0

In [13]:
df = pd.read_csv(EVAL_FILE)
len(df)

500

In [14]:
def build_prompt(headline: str) -> str:
    return (
        f"{headline.strip()}\n\n"
        "Tarefa: classificar a manchete como impacto 'em alta' ou 'em baixa' no real brasileiro. "
        "Saída autorizada: apenas um destes dois rótulos. "
        "Saídas não conformes (qualquer conteúdo adicional) são inválidas."
    )

In [15]:
# === Step 3: Query Ollama ===
def query_ollama(model, prompt):
    res = requests.post("http://localhost:11434/api/chat", json={
        "model": model,
        "temperature": TEMPERATURE,
        "stream": False,
        "messages": [{"role": "user", "content": prompt}]
    })
    res.raise_for_status()
    return res.json()["message"]["content"].strip()

In [16]:
# === Step 4: Run prediction loop ===
predictions = []
for i, row in df.iterrows():
    prompt = build_prompt(row["Headline"])
    try:
        out = query_ollama(MODEL, prompt)
    except Exception as e:
        out = "Erro"
    predictions.append(out)
    time.sleep(0.25)  # reduce overload risk

In [17]:
import re
predictions = [
    re.sub(r'\s*\n.*$', '', p)           # remove anything after newline
    .strip()                             # trim spaces
    .strip("'\"")                        # strip leading/trailing quotes
    for p in predictions
]

In [18]:
predictions

['Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em alta',
 'Em alta',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em alta',
 'Em alta',
 'Em baixa',
 'Em alta',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em alta',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em alta',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em alta',
 'Em baixa',
 'Em alta',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em alta',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em alta',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em alta',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em alta',
 'Em alta',
 'Em baixa',
 'Em alta',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 

In [19]:
df["Prediction"] = predictions
df.to_csv(RESULT_FILE, index=False)
print(f"Saved predictions to {RESULT_FILE}")

Saved predictions to ../../results/exp002/results_filtered.csv


### Predictions

In [21]:
from sklearn.metrics import classification_report

# Reload the results file
df = pd.read_csv("../../results/exp002/results_filtered.csv")

df['Prediction'] = df['Prediction'].map({
    'Em baixa': 'Increase',
    'Em alta': 'Decrease'
})

# Report
report = classification_report(
    df["Direction"], df["Prediction"],
    labels=["Increase", "Decrease"],
    target_names=["Increase", "Decrease"],
    digits=3
)

print("\nClassification Report:\n")
print(report)


Classification Report:

              precision    recall  f1-score   support

    Increase      0.524     0.796     0.632       265
    Decrease      0.443     0.183     0.259       235

    accuracy                          0.508       500
   macro avg      0.483     0.490     0.445       500
weighted avg      0.486     0.508     0.457       500



In [22]:
from sklearn.metrics import confusion_matrix
import pandas as pd

labels = ["Increase", "Decrease"]

cm = confusion_matrix(df["Direction"], df["Prediction"], labels=labels)
cm_df = pd.DataFrame(
    cm,
    index=[f"Actual {label}" for label in labels],
    columns=[f"Predicted {label}" for label in labels]
)

print("\nConfusion Matrix:\n")
display(cm_df)



Confusion Matrix:



,Predicted Increase,Predicted Decrease
Actual Increase,211,54
Actual Decrease,192,43
